In [6]:
!pip install !pip install tensorflow_io
!pip install keras-tuner
!pip install print_schema
!pip install pydub
!pip install opensmile
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install transformers

ERROR: Invalid requirement: '!pip'
  Using cached keras_tuner-1.1.3-py3-none-any.whl (135 kB)
  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Using cached protobuf-3.19.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached Werkzeug-2.1.2-py3-none-any.whl (224 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'message.py'
Consider using the `--user` option or check the permissions.

  Using cached print_schema-1.1.1-py3-none-any.whl (6.2 kB)
  Using cached pyd

In [1]:
import numpy as np
import pandas as pd
import gc

In [2]:
data = pd.read_csv("train.csv")
id = data.iloc[:,0].values
train_labels = data.iloc[:,1].values
train_texts = data.iloc[:,2].values

In [3]:
df = pd.read_csv("test.csv")
df["Content"].fillna("",inplace=True)
test_texts = list(df.loc[:,"Content"])
test_labels = list(df.loc[:,"Label"])

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
def get_several_validation_set(content, label, val_fraction=0.2, total_splits=5, seed=0):
  sss = StratifiedShuffleSplit(n_splits=total_splits, test_size=val_fraction, random_state=seed)
  return sss.split(content, label)

In [7]:
import torch
import torchvision
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in /home/jupyter/.cache/torch/hub/huggingface_pytorch-transformers_main


In [8]:
class ADdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [9]:
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 256, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            
            nn.Linear(D_in, 256),
            nn.BatchNorm1d(num_features=256), ## kk
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            #nn.BatchNorm1d(num_features=128),
            #nn.ReLU(),
            #nn.Linear(128, 1)
            
            #nn.BatchNorm1d(num_features=128), ###
            #nn.ReLU(),
            
            #nn.Dropout(0.5),

            #nn.Linear(H, D_out)
            #nn.Linear(128, 2)
            #nn.Linear(128, 1)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [10]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader


def initialize_model(epochs=50):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_loader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [11]:
import random
import time

# Specify loss function
#loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.BCELoss()
loss_fn =  torch.nn.BCEWithLogitsLoss()
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_loader, optim, val_loader=None, epochs=50, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = [] 
    for epoch_i in range(epochs):
        train_loss_sum = 0
        train_accuracy_epoch = 0
        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for batch in (train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Perform a forward pass. This will return logits.
            logits = model(input_ids, attention_mask)

            # Compute loss and accumulate the loss values
            logits = logits.reshape(-1) #silebilirsin
            
            loss = loss_fn(logits, labels.float())
            train_loss_sum += loss.item()

            logits_class = logits > 0.5
            train_acc = (labels == logits_class).sum().item() / labels.size(0)
            train_accuracy_epoch += train_acc

            # Perform a backward pass to calculate gradients
            loss.backward()


            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

        avg_train_loss = np.round(train_loss_sum/len(train_loader),2)
        avg_train_acc = np.round(train_accuracy_epoch/len(train_loader),2)

        if evaluation == True:
            avg_val_loss, avg_val_acc = evaluate(model, val_loader)
        # print('Epoch {}, train loss {} , val loss is {}, train acc is {}, val acc is {} '.format(epoch_i,avg_train_loss,avg_val_loss,avg_train_acc,avg_val_acc))
        train_loss_list.append(avg_train_loss)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        train_acc_list.append(avg_train_acc)

    print("Training complete!")
    return model,train_loss_list,val_loss_list,train_acc_list,val_acc_list


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_loss_sum = 0
    val_accuracy_epoch = 0
    # For each batch in our validation set...
    for batch in val_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        val_loss_sum +=loss.item()
        avg_val_loss = np.round(val_loss_sum/len(val_dataloader),2)

        logits_class = logits > 0.5
        val_acc = (labels == logits_class).sum().item() / labels.size(0)
        val_accuracy_epoch += val_acc
        avg_val_acc = np.round(val_accuracy_epoch/len(val_dataloader),2)

    return avg_val_loss, avg_val_acc


def evaluate_test(model, test_dataloader):
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    test_loss_sum = 0
    test_accuracy_epoch = 0
    predictions = []
    labels_list = []
    for batch in test_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)
        labels_list.append(labels.cpu())

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        test_loss_sum +=loss.item()
        avg_test_loss = np.round(test_loss_sum/len(test_dataloader),5)

        logits_class = logits > 0.5
        predictions.append(logits.cpu())
        test_acc = (labels == logits_class).sum().item() / labels.size(0)
        test_accuracy_epoch += test_acc
        avg_test_acc = np.round(test_accuracy_epoch/len(test_dataloader),5)

    return avg_test_loss, avg_test_acc, predictions, labels_list

In [12]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = ADdataset(test_encodings, test_labels)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [13]:
# seeds = list(range(0,60,6))
mean_val_acc = {}
max_val_acc = {}
test_acc = {}
best_val_acc = 0
best_test_acc = 0

In [14]:
gc.collect()

121

In [15]:
#val_fraction = 0.2
gc.collect()
set_seed(42)    # Set seed for reproducibility
seed = 42
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
mean_val_acc[seed] = []
last_val_acc[seed] = []
test_acc[seed] = []
models = []
index = 1
train_and_val_splits = get_several_validation_set(train_texts, train_labels, total_splits=10, seed=seed)
for train_index, val_index in train_and_val_splits:
    gc.collect()
    train_encodings = tokenizer(list(train_texts[train_index]), truncation=True, padding=True)
    val_encodings = tokenizer(list(train_texts[val_index]), truncation=True, padding=True)
    train_dataset = ADdataset(train_encodings, train_labels[train_index])
    val_dataset = ADdataset(val_encodings, train_labels[val_index])
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
    if index == 1:
        bert_classifier, optimizer, scheduler = initialize_model(epochs=20)
    else:
        bert_classifier = torch.load(f"weights/iteration/Model_{index-1}.pt")
    bert_classifier,train_loss_list,val_loss_list,train_acc_list,val_acc_list = train(bert_classifier, train_loader, optimizer, val_loader, epochs=20, evaluation=True)

    max_val_accuracy = max(val_acc_list)
    if max_val_accuracy > best_val_acc:
        best_model_val = bert_classifier
        torch.save(best_model_val, "weights/iteration/bestModel_val.pt")


    test_loss, test_accuracy, predictions, labels_l = evaluate_test(bert_classifier, test_loader)
    print("Test Loss: {}, Test Accuracy: {}".format(test_loss,test_accuracy))

    if test_accuracy > best_test_acc:
        best_model_test = bert_classifier
        torch.save(best_model_test, "weights/iteration/bestModel_test.pt")
    torch.save(bert_classifier, f"weights/iteration/Model_{index}.pt")
    models.append(bert_classifier)
    index += 1
    

    mean_val_acc[seed].append(np.mean(val_acc_list))
    last_val_acc[seed].append(val_acc_list[-1])
    test_acc[seed].append(test_accuracy)
    gc.collect()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Start training...

Training complete!
Test Loss: 0.53905, Test Accuracy: 0.84524
Start training...

Training complete!
Test Loss: 0.5242, Test Accuracy: 0.84524
Start training...

Training complete!
Test Loss: 0.53183, Test Accuracy: 0.84524
Start training...

Training complete!
Test Loss: 0.53108, Test Accuracy: 0.85913
Start training...

Training complete!
Test Loss: 0.52917, Test Accuracy: 0.84524
Start training...

Training complete!
Test Loss: 0.52737, Test Accuracy: 0.85913
Start training...

Training complete!
Test Loss: 0.56069, Test Accuracy: 0.84524
Start training...

Training complete!
Test Loss: 0.54097, Test Accuracy: 0.84524
Start training...

Training complete!
Test Loss: 0.55027, Test Accuracy: 0.85913
Start training...



RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 14.76 GiB total capacity; 13.47 GiB already allocated; 27.75 MiB free; 13.66 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
print(mean_val_acc)
print(last_val_acc)
print(test_acc)

{42: [0.8564999999999999, 1.0, 0.9800000000000001, 1.0, 1.0, 0.976, 0.9429999999999998, 0.9800000000000001, 1.0]}
{42: [0.9, 1.0, 0.98, 1.0, 1.0, 0.98, 0.95, 0.98, 1.0]}
{42: [0.84524, 0.84524, 0.84524, 0.85913, 0.84524, 0.85913, 0.84524, 0.84524, 0.85913]}


In [22]:
#val_fraction = 0.1
gc.collect()
set_seed(42)    # Set seed for reproducibility
seed = 42
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
mean_val_acc[seed] = []
max_val_acc[seed] = []
test_acc[seed] = []
models = []
index = 1
train_and_val_splits = get_several_validation_set(train_texts, train_labels, val_fraction = 0.1, total_splits=10, seed=seed)
for train_index, val_index in train_and_val_splits:
    gc.collect()
    train_encodings = tokenizer(list(train_texts[train_index]), truncation=True, padding=True)
    val_encodings = tokenizer(list(train_texts[val_index]), truncation=True, padding=True)
    train_dataset = ADdataset(train_encodings, train_labels[train_index])
    val_dataset = ADdataset(val_encodings, train_labels[val_index])
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
    if index == 1:
        bert_classifier, optimizer, scheduler = initialize_model(epochs=20)
    else:
        bert_classifier = torch.load(f"weights/iteration/Model_0.1_{index-1}.pt")
    bert_classifier,train_loss_list,val_loss_list,train_acc_list,val_acc_list = train(bert_classifier, train_loader, optimizer, val_loader, epochs=20, evaluation=True)

    max_val_accuracy = max(val_acc_list)
    if max_val_accuracy > best_val_acc:
        best_model_val = bert_classifier
        torch.save(best_model_val, "weights/iteration/bestModel_0.1_val.pt")


    test_loss, test_accuracy, predictions, labels_l = evaluate_test(bert_classifier, test_loader)
    print("Test Loss: {}, Test Accuracy: {}".format(test_loss,test_accuracy))

    if test_accuracy > best_test_acc:
        best_model_test = bert_classifier
        torch.save(best_model_test, "weights/iteration/bestModel_0.1_test.pt")
    torch.save(bert_classifier, f"weights/iteration/Model_0.1_{index}.pt")
    models.append(bert_classifier)
    index += 1
    

    mean_val_acc[seed].append(np.mean(val_acc_list))
    max_val_acc[seed].append(max_val_accuracy)
    test_acc[seed].append(test_accuracy)
    gc.collect()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Start training...

Training complete!
Test Loss: 0.56648, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.5537, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.55782, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.54253, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.57136, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.57855, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.54912, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.54856, Test Accuracy: 0.87302
Start training...

Training complete!
Test Loss: 0.55747, Test Accuracy: 0.87302
Start training...



RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 14.76 GiB total capacity; 13.43 GiB already allocated; 75.75 MiB free; 13.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [24]:
print(mean_val_acc)
print(max_val_acc)
print(test_acc)

{42: [0.7989999999999999, 1.0, 1.0, 0.9600000000000002, 1.0, 1.0, 1.0, 1.0, 0.9455000000000002]}
{42: [0.92, 1.0, 1.0, 0.96, 1.0, 1.0, 1.0, 1.0, 0.96]}
{42: [0.87302, 0.87302, 0.87302, 0.87302, 0.87302, 0.87302, 0.87302, 0.87302, 0.87302]}
